In [11]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from base64 import b64decode
from os import urandom

In [12]:
class EncryptionOracleInterface:
    def encrypt(self, data: bytes) -> bytes:
        pass

In [13]:
def ECB_CBC_detector(oracle: type[EncryptionOracleInterface]) -> str:
    data = b"A" * 48
    ct = oracle.encrypt(data)

    blocks = [ct[i : i + AES.block_size] for i in range(0, len(ct), AES.block_size)]

    return "ECB" if len(blocks) != len(set(blocks)) else "CBC"

In [14]:
class encryptionOracle(EncryptionOracleInterface):
    def __init__(self):
        self.key = urandom(16)
        self.cipher = AES.new(self.key, AES.MODE_ECB)
        self.secret = b64decode(
            "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
        )

    def encrypt(self, data: bytes) -> bytes:
        pt = data + self.secret

        return self.cipher.encrypt(pad(pt, AES.block_size))

In [15]:
def get_block_size(oracle: type[EncryptionOracleInterface]) -> int:
    test = b""
    base_len = len(oracle.encrypt(test))
    new_len = base_len

    while new_len == base_len:
        test += b"A"
        new_len = len(oracle.encrypt(test))

    return new_len - base_len

In [16]:
def recover_secret(oracle: type[EncryptionOracleInterface], secret_size: int) -> bytes:
    assert ECB_CBC_detector(oracle) == "ECB"

    feed = b"A" * secret_size
    recovered = b""
    for i in range(secret_size):
        pt = feed[:-i]
        ct = oracle.encrypt(pt)
        to_match = ct[:secret_size]
        for b in range(256):
            test = pt + recovered + bytes([b])
            if oracle.encrypt(test)[:secret_size] == to_match:
                recovered += bytes([b])
                break

    return recovered

In [17]:
oracle = encryptionOracle()
secret_size = len(oracle.encrypt(b''))
secret = recover_secret(oracle, secret_size)

print(secret.decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

